In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

class Graph:
    def __init__(self):
        self.edges = {}

    def add_edge(self, from_node, to_node, weight):
        if from_node not in self.edges:
            self.edges[from_node] = []
        if to_node not in self.edges:
            self.edges[to_node] = []  # Ensure the node exists in the graph
        for idx, (neighbor, _) in enumerate(self.edges[from_node]):
            if neighbor == to_node:
                self.edges[from_node][idx] = (to_node, weight)  # Update existing weight
                break
        else:
            self.edges[from_node].append((to_node, weight))  # Add new edge
        for idx, (neighbor, _) in enumerate(self.edges[to_node]):
            if neighbor == from_node:
                self.edges[to_node][idx] = (from_node, weight)  # Update existing weight
                break
        else:
            self.edges[to_node].append((from_node, weight))  # Add reverse edge

    def dijkstra(self, start, end):
        distances = {node: float('infinity') for node in self.edges}
        distances[start] = 0
        previous_nodes = {node: None for node in self.edges}
        unvisited = list(self.edges.keys())

        while unvisited:
            current_node = min(unvisited, key=lambda node: distances[node])

            if distances[current_node] == float('infinity'):
                break  # All remaining nodes are unreachable

            unvisited.remove(current_node)

            for neighbor, weight in self.edges.get(current_node, []):
                distance = distances[current_node] + weight

                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_nodes[neighbor] = current_node

        return distances, previous_nodes

    def shortest_path(self, start, end):
        distances, previous_nodes = self.dijkstra(start, end)
        path = []
        current_node = end

        while current_node is not None:
            path.append(current_node)
            current_node = previous_nodes[current_node]

        path.reverse()
        return path, distances[end]

    def triangular_layout(self):
        pos = {}
        if len(self.edges) == 3:
            pos[list(self.edges.keys())[0]] = (0, 1)  # Top vertex
            pos[list(self.edges.keys())[1]] = (-1, -1)  # Bottom left vertex
            pos[list(self.edges.keys())[2]] = (1, -1)  # Bottom right vertex
        else:
            pos = nx.spring_layout(nx.Graph(self.edges))

        return pos

    def draw_graph(self):
        G = nx.Graph()
        
        for from_node, edges in self.edges.items():
            for to_node, weight in edges:
                G.add_edge(from_node, to_node, weight=weight)

        pos = self.triangular_layout()
        nx.draw(G, pos, with_labels=True, node_size=2000, node_color='skyblue', font_size=10, font_weight='bold')
        
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        
        plt.title("Airplane Travel Routes")
        plt.show()

    def draw_shortest_path(self, path):
        G = nx.DiGraph()
        
        for from_node, edges in self.edges.items():
            for to_node, weight in edges:
                G.add_edge(from_node, to_node, weight=weight)

        pos = self.triangular_layout()
        nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightgreen', font_size=10, font_weight='bold')
        
        edge_labels = nx.get_edge_attributes(G, 'weight')
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
        
        path_edges = [(path[i], path[i + 1]) for i in range(len(path) - 1)]
        nx.draw_networkx_edges(G, pos, edgelist=path_edges, arrowstyle='-|>', arrowsize=20, width=2, edge_color='orange')

        plt.title(f"Shortest Path from {path[0]} to {path[-1]}")
        plt.show()

    def all_routes(self, start):
        routes = []
        visited = set()

        def backtrack(current_route):
            if current_route:
                routes.append(current_route.copy())  # Add the current route to the list
            for neighbor in self.edges.get(current_route[-1], []):
                if neighbor[0] not in visited:
                    visited.add(neighbor[0])
                    current_route.append(neighbor[0])
                    backtrack(current_route)
                    current_route.pop()  # Remove the last city
                    visited.remove(neighbor[0])  # Backtrack

        visited.add(start)
        backtrack([start])
        return routes

    def calculate_route_distance(self, route):
        total_distance = 0
        for i in range(len(route) - 1):
            for neighbor, weight in self.edges.get(route[i], []):
                if neighbor == route[i + 1]:
                    total_distance += weight
                    break
        return total_distance


# Sample usage
if __name__ == "__main__":
    graph = Graph()
    
    # Input edges from the user
    while True:
        from_city = input("Enter the starting city (or 'done' to finish): ")
        if from_city.lower() == 'done':
            break
        to_city = input("Enter the destination city: ")
        weight = float(input("Enter the Distance: "))
        graph.add_edge(from_city, to_city, weight)

    # Draw the graph after user input
    graph.draw_graph()

    # Get shortest path input
    start_city = input("Enter the starting city for shortest path calculation: ")
    end_city = input("Enter the destination city: ")

    # Validate if the input cities are in the graph
    if start_city not in graph.edges or end_city not in graph.edges:
        print(f"Invalid input. Please ensure both cities are in the graph.")
    else:
        path, distance = graph.shortest_path(start_city, end_city)

        if distance == float('infinity'):
            print(f"No path exists from {start_city} to {end_city}.")
        else:
            print(f"The shortest path from {start_city} to {end_city} is: {' -> '.join(path)} with a distance of {distance}.")
            graph.draw_shortest_path(path)

    # Generate all routes starting from a specified city
    start_city_for_routes = input("Enter the starting city to get all possible routes: ")
    if start_city_for_routes in graph.edges:
        routes = graph.all_routes(start_city_for_routes)
        print("All possible routes starting from", start_city_for_routes, ":")
        for route in routes:
            route_distance = graph.calculate_route_distance(route)
            print(f"{' -> '.join(route)} with a distance of {route_distance}")
    else:
        print(f"{start_city_for_routes} is not in the graph.")
